In [ ]:
import sys
import networkx as nx
import pickle
import itertools as it
import os
import shutil

import MDAnalysis as mda
import MDAnalysis.transformations as trans
import nglview as nv


import warnings
warnings.filterwarnings(action='once')


if '/home/anna/Documents/cinnabar' not in sys.path:
    sys.path.insert(1, '/home/anna/Documents/cinnabar')
import cinnabar

print(cinnabar.__file__)

print("adding code to the pythonpath...")
code = '/home/anna/Documents/code/python'
if code not in sys.path:
    sys.path.insert(1, code)
import pipeline

print(pipeline.__file__)

from pipeline import *
from pipeline.analysis import *
from pipeline.prep import *
from pipeline.utils import *


In [ ]:
transf = ["lig_ejm31~lig_ejm45", "lig_ejm44~lig_ejm45","lig_ejm44~lig_ejm49"]
engine = "GROMACS"
main_dir = "/home/anna/Documents/benchmark/tyk2_benchmark"
methods = ["1fs", "2fs_HMR4", "4fs_HMR4", "4fs_HMR3", "2fs_HMR3", "2fs"]
# methods = ["2fs"]

# options
ana_file = f"{main_dir}/execution_model/analysis_protocol.dat"
analysis_options = analysis_protocol(ana_file, auto_validate=True)
# analysis_options.rewrite_protocol()

for method in methods:
    for trans in transf:
        path_to_dir = f"{main_dir}/outputs_extracted/{engine}/{method}/{trans}"
        print(path_to_dir)
        final_results_folder = f"{main_dir}/outputs_extracted/results"

        try:
            # using the pipeline module for analysis
            analysed_pert = analyse(path_to_dir)
            analysed_pert.set_options(analysis_options)
            # analysed_pert.set_options({"try pickle":False})
            avg, error, repeats_tuple_list = analysed_pert.analyse_all_repeats()
            # analysed_pert.plot_graphs()
            write_analysis_file(analysed_pert, final_results_folder, method=method)
        except Exception as e:
            print(e)
            print(f"could not analyse {path_to_dir}")

In [ ]:
transf = ["lig_ejm31~lig_ejm45", "lig_ejm31~lig_ejm48","lig_ejm43~lig_ejm46", "lig_ejm46~lig_jmc28", "lig_jmc27~lig_jmc28"]
engine = "AMBER"
main_dir = "/home/anna/Documents/benchmark/tyk2_benchmark"
# methods = ["1fs", "2fs_HMR4", "4fs", "4fs_HMR3", "2fs_HMR3", "2fs"]
methods = ["new_mapping"]

# options
ana_file = f"{main_dir}/execution_model/analysis_protocol.dat"
analysis_options = analysis_protocol(ana_file, auto_validate=True)
# analysis_options.rewrite_protocol()

for method in methods:
    for trans in transf:
        path_to_dir = f"{main_dir}/outputs_extracted/{engine}/{method}/{trans}"
        print(path_to_dir)
        final_results_folder = f"{main_dir}/outputs_extracted/results"

        try:
            # using the pipeline module for analysis
            analysed_pert = analyse(path_to_dir)
            analysed_pert.set_options(analysis_options)
            avg, error, repeats_tuple_list = analysed_pert.analyse_all_repeats()
            # analysed_pert.plot_graphs()
            write_analysis_file(analysed_pert, final_results_folder, method=method)
        except Exception as e:
            print(e)
            print(f"could not analyse {path_to_dir}")

In [ ]:
protein = "mcl1"

bench_folder = f"/home/anna/Documents/benchmark"
main_dir = f"{bench_folder}/extracted/{protein}"

# choose location for the files
net_file = f"{main_dir}/execution_model/network_lomap.dat"
ana_file = f"{main_dir}/execution_model/analysis_protocol.dat"
# exp_file = f"{bench_folder}/inputs/experimental/tyk2.yml"
exp_file = f"/home/anna/Documents/benchmark/inputs/experimental/{protein}.yml"

if os.path.exists(f"{main_dir}/outputs_extracted/results"):
    results_folder = f"{main_dir}/outputs_extracted/results"
elif os.path.exists(f"{main_dir}/outputs/results"):
    results_folder = f"{main_dir}/outputs/results"
else:
    raise ValueError(f"results directory not found in the {main_dir}. please make sure results were written using the analysis script previously in the pipeline")

output_folder = validate.folder_path(f"{main_dir}/analysis", create=True)

all_analysis_object = analysis_network(results_folder,
                                       exp_file=exp_file,
                                       net_file=net_file,
                                       output_folder=output_folder,
                                       analysis_ext=ana_file
                                        )

# can add any other results files
all_analysis_object.compute(cycle_closure=False)
# all_analysis_object.compute_other_results(file_name=None, name=None)
all_analysis_object.compute_other_results(sorted(glob.glob(f"{bench_folder}/inputs/other_computed/exscentia/*.csv")), name="Exscientia")

eng = "AMBER"
perturbations = ["lig_30~lig_40", "lig_27~lig_48"]
# ['lig_ejm42~lig_ejm44', 'lig_ejm42~lig_ejm49', 'lig_ejm42~lig_ejm45']
# ['lig_ejm42~lig_ejm49', 'lig_ejm45~lig_ejm55', 'lig_ejm43~lig_ejm50'] #gromacs
# ['lig_ejm45~lig_ejm53', 'lig_ejm44~lig_ejm45', 'lig_ejm31~lig_ejm49'] #somd

for pert in perturbations:
    try:
        all_analysis_object.remove_perturbations(pert)
    except:
        pass

all_analysis_object.compute()

In [ ]:
values = all_analysis_object.compute_mue(pert_val="pert", engines="AMBER")
print(values)

In [ ]:
all_analysis_object._plotting_object.outlier(pert_val="pert", engines="AMBER", name="Exscientia", outliers=5)

In [ ]:
all_analysis_object._initalise_stats_object()
all_analysis_object._stats_object.compute_mue(pert_val="pert", y="Exscientia", x="AMBER")

In [ ]:
all_analysis_object.calc_mae(pert_val="pert")